In [1]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [2]:
!unzip '/content/gdrive/My Drive/hyperspectral_reconstruction_project/retina_data.zip'

Archive:  /content/gdrive/My Drive/hyperspectral_reconstruction_project/retina_data.zip
replace trainLabels.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [3]:
import matplotlib.pyplot as plt
import matplotlib.image as img
import tensorflow.keras as tf
import numpy as np

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

datasetdir = r'./train_imagefolder/'

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 32
shape = (512, 482)

def generators(): 
    '''Create the training and validation datasets'''
    
    imgdatagen = ImageDataGenerator(
        validation_split = 0.2,
    )


    train_dataset = imgdatagen.flow_from_directory(
        directory = datasetdir,
        target_size = shape,
        class_mode = "categorical",
        batch_size = batch_size,
        subset = 'training', 
        shuffle = True,
        seed = 42,
        keep_aspect_ratio=False
    )

    val_dataset = imgdatagen.flow_from_directory(
        directory = datasetdir,
        target_size = shape,
        class_mode = "categorical",
        batch_size = batch_size,
        subset = 'validation',
        shuffle = True,
        seed = 42,
        keep_aspect_ratio=False
    )
    return train_dataset, val_dataset

In [5]:
# resnet_model = Sequential()
# from tensorflow.keras.applications.resnet50 import ResNet50

# pretrained_model= ResNet50(include_top = False,
#                    input_shape = (512, 482, 3),
#                    pooling = 'avg', classes = 5,
#                    weights = None)

# for layer in pretrained_model.layers:
#         layer.trainable = False

# resnet_model.add(pretrained_model)
# resnet_model.add(Flatten())
# resnet_model.add(Dense(units = 512, activation='relu'))
# resnet_model.add(Dropout(0.5))
# resnet_model.add(Dense(units = 5, activation='softmax'))

# resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 5)                 2565      
                                                                 
Total params: 24,639,365
Trainable params: 1,051,653
Non-trainable params: 23,587,712
_________________________________________________________________


In [6]:

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(512, 482, 3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [7]:
train_dataset, val_dataset = generators()

Found 28103 images belonging to 5 classes.
Found 7023 images belonging to 5 classes.


In [8]:
from sklearn.utils import class_weight
import numpy as np

class_weights = class_weight.compute_class_weight(class_weight = 'balanced',
            classes = np.unique(train_dataset.classes), 
            y = train_dataset.classes)

In [9]:
model.compile(optimizer=Adam(lr=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])
history = model.fit(train_dataset, validation_data = val_dataset, 
                           class_weight = {i: weight for i, weight in enumerate(class_weights)}, 
                           epochs = 1, shuffle = True)

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


879/879 [==============================] - ETA: 0s - loss: 20.6358 - accuracy: 0.0788

KeyboardInterrupt: ignored

In [ ]:
class_weights

In [ ]:
# lr
resnet_model.compile(optimizer=Adam(lr = 4), loss='categorical_crossentropy', metrics = ['accuracy'])

history = resnet_model.fit(train_dataset, validation_data = val_dataset, 
                           class_weight = {i: weight for i, weight in enumerate(class_weights)}, 
                           epochs = 5, shuffle = True)

In [ ]:
pred_val = resnet_model.predict(val_dataset)

In [ ]:
pred_val[:20]

In [ ]:
from sklearn.metrics import classification_report
pred_val_max = np.argmax(pred_val, axis=1)

print(classification_report(val_dataset.labels, pred_val_max, target_names = [str(i) for i in range(5)]))

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(val_dataset.labels, pred_val_max, labels = [str(i) for i in range(5)]))

In [ ]:
len(np.unique(pred_val_max))

In [ ]:
import tensorflow as tf

datasets = [tf.data.Dataset.list_files(f'train_imagefolder/{i}/*.jpeg').take(700) for i in range(4)]

res_dataset = datasets[0]
res_dataset = [res_dataset.concatenate(ds) for ds in datasets[1:]]
res_dataset.__len__
# ds = dogs.concatenate(cats).concatenate(turtles).shuffle(3000)

In [ ]:
def get_label(file_path):
  parts = tf.strings.split(file_path, os.path.sep)
  one_hot = parts[-2] == class_names
  return tf.argmax(one_hot)

def decode_img(img):
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.image.resize(img, [img_height, img_width])
  return img

def process_path(file_path):
  label = get_label(file_path)
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label


train_ds = res_dataset.map(process_path)